# Pandas - GroupBy a transformace DataFrames

## Úvod do GroupBy a transformací DataFrames

Transformace DataFrames umožňují reorganizovat a přeformátovat data pro lepší analýzu a vizualizaci. Pandas nabízí několik klíčových metod pro práci se strukturovanými daty.

### Kdy použít kterou metodu

| Metoda | Účel | Ideální použití |
|--------|------|-----------------|
| `groupby()` | Agregace podle skupin | Výpočet statistik podle kategorií |
| `pivot()` | Změna formátu dat | Přeformátování pro analýzu nebo vizualizaci |
| `pivot_table()` | Pivot s agregací | Souhrnné tabulky s více hodnotami |
| `transpose()` | Výměna os | Změna orientace tabulky |

## Přehled metod GroupBy a transformací

| Metoda | Popis | Příklad |
|--------|-------|---------|
| `df.groupby()` | Seskupení dat podle hodnot | `df.groupby('rok')` |
| `grouped.agg()` | Aplikace agregačních funkcí | `grouped.agg(['mean', 'sum'])` |
| `grouped.transform()` | Transformace zachovávající velikost | `grouped.transform('mean')` |
| `grouped.groups` | Zobrazení skupin | `grouped.groups` |
| `grouped.get_group()` | Získání konkrétní skupiny | `grouped.get_group(2021)` |
| `df.pivot()` | Přeformátování na široký formát | `df.pivot(index='rok', columns='mesic')` |
| `df.pivot_table()` | Pivot s agregací | `df.pivot_table(values='srazky', aggfunc='mean')` |
| `df.transpose()` / `df.T` | Otočení DataFrame | `df.T` |
| `df.set_index()` | Nastavení indexu | `df.set_index('rok')` |
| `df.reset_index()` | Reset indexu | `df.reset_index()` |
| `df.fillna()` | Vyplnění chybějících hodnot | `df.fillna(0)` |

In [24]:
import pandas as pd
import numpy as np

# Základní meteorologická data
data_zaklad = {
    'rok': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022],
    'mesic': [6, 7, 8, 6, 7, 8, 6, 7, 8],
    'srazky': [45.2, 38.7, 52.1, 41.3, 55.8, 48.9, 39.7, 42.1, 46.3],
    'teplota': [18.5, 22.1, 21.8, 19.2, 23.4, 22.6, 20.1, 24.7, 23.2]
}
df_zaklad = pd.DataFrame(data_zaklad)
print("Základní meteorologická data:")
print(df_zaklad)

Základní meteorologická data:
    rok  mesic  srazky  teplota
0  2020      6    45.2     18.5
1  2020      7    38.7     22.1
2  2020      8    52.1     21.8
3  2021      6    41.3     19.2
4  2021      7    55.8     23.4
5  2021      8    48.9     22.6
6  2022      6    39.7     20.1
7  2022      7    42.1     24.7
8  2022      8    46.3     23.2


## GroupBy operace

### Základní skupinování

GroupBy operace umožňují rozdělit data do skupin podle hodnot ve sloupcích a následně aplikovat agregační funkce.

In [25]:
# Data pro základní skupinování
data_skupiny = {
    'rok': [2020, 2020, 2021, 2021, 2022, 2022, 2023, 2023],
    'mesic': [3, 9, 3, 9, 3, 9, 3, 9],
    'srazky': [28.5, 67.2, 31.7, 72.8, 29.1, 65.4, 33.2, 69.7]
}
df_skupiny = pd.DataFrame(data_skupiny)

# Vytvoření GroupBy objektu
skupiny_podle_roku = df_skupiny.groupby('rok')

# Základní agregace
prumer_podle_roku = skupiny_podle_roku['srazky'].mean()
soucet_podle_roku = skupiny_podle_roku['srazky'].sum()
pocet_podle_roku = skupiny_podle_roku['srazky'].count()

print("Průměr srážek podle roku:")
print(prumer_podle_roku)
print("\nSoučet srážek podle roku:")
print(soucet_podle_roku)
print("\nPočet měření podle roku:")
print(pocet_podle_roku)

Průměr srážek podle roku:
rok
2020    47.85
2021    52.25
2022    47.25
2023    51.45
Name: srazky, dtype: float64

Součet srážek podle roku:
rok
2020     95.7
2021    104.5
2022     94.5
2023    102.9
Name: srazky, dtype: float64

Počet měření podle roku:
rok
2020    2
2021    2
2022    2
2023    2
Name: srazky, dtype: int64


### Přístup ke skupinám

In [26]:
# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření dat o srážkách za dva roky
# Každý rok má měření za 3 měsíce (duben, květen, červen)
data_pristup = {
   'rok': [2021, 2021, 2021, 2022, 2022, 2022],
   'mesic': [4, 5, 6, 4, 5, 6],
   'srazky': [32.4, 41.7, 45.8, 28.9, 38.2, 42.3]
}

# Převod na DataFrame
df_pristup = pd.DataFrame(data_pristup)

# ----- 2. VYTVOŘENÍ SKUPIN -----
# groupby() vytvoří objekt GroupBy, který rozdělí data podle hodnot ve sloupci 'rok'
# Data se fyzicky nerozdělí, pouze se vytvoří "návod" jak k nim přistupovat
skupiny_pristup = df_pristup.groupby('rok')

# ----- 3. ZOBRAZENÍ NÁZVŮ SKUPIN -----
# groups.keys() vrací všechny unikátní hodnoty, podle kterých jsme seskupili
# V našem případě: 2021 a 2022
print("Názvy skupin:")
print(list(skupiny_pristup.groups.keys()))

# ----- 4. PŘÍSTUP KE KONKRÉTNÍ SKUPINĚ -----
# get_group() umožňuje získat všechny řádky patřící do jedné skupiny
# Zde získáme všechna data pro rok 2021
skupina_2021 = skupiny_pristup.get_group(2021)
print("\nData pro rok 2021:")
print(skupina_2021)

# ----- 5. ITERACE PŘES VŠECHNY SKUPINY -----
# GroupBy objekt lze procházet cyklem for
# Každá iterace vrátí dvojici: (název skupiny, DataFrame se daty skupiny)
print("\nIterace přes všechny skupiny:")
for nazev_roku, data_roku in skupiny_pristup:
   print(f"Rok {nazev_roku}:")              # Vypíše název skupiny (rok)
   print(data_roku[['mesic', 'srazky']])    # Vypíše pouze vybrané sloupce
   print()                                   # Prázdný řádek pro oddělení

# Při iteraci postupně projdeme:
# 1. iterace: nazev_roku = 2021, data_roku = DataFrame s 3 řádky pro rok 2021
# 2. iterace: nazev_roku = 2022, data_roku = DataFrame s 3 řádky pro rok 2022

Názvy skupin:
[2021, 2022]

Data pro rok 2021:
    rok  mesic  srazky
0  2021      4    32.4
1  2021      5    41.7
2  2021      6    45.8

Iterace přes všechny skupiny:
Rok 2021:
   mesic  srazky
0      4    32.4
1      5    41.7
2      6    45.8

Rok 2022:
   mesic  srazky
3      4    28.9
4      5    38.2
5      6    42.3



### Agregační funkce

Metoda `agg()` umožňuje aplikovat více agregačních funkcí současně a pojmenovat výsledné sloupce.

In [27]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření slovníku s daty o srážkách za 3 roky
# Každý rok má měření ze 3 měsíců (leden, červen, prosinec)
data_agregace = {
   'rok': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022],
   'mesic': [1, 6, 12, 1, 6, 12, 1, 6, 12],
   'srazky': [15.2, 45.8, 23.1, 18.7, 52.3, 28.4, 16.9, 48.1, 25.7]
}

# Převod slovníku na DataFrame pro snadnější práci s daty
df_agregace = pd.DataFrame(data_agregace)

# ----- 2. VÍCE AGREGAČNÍCH FUNKCÍ SOUČASNĚ -----
# Seskupíme data podle roku a na sloupec 'srazky' aplikujeme více funkcí najednou
# agg() umožňuje použít seznam funkcí, které se mají vypočítat
vysledek_vice_funkci = df_agregace.groupby('rok')['srazky'].agg(['mean', 'sum', 'std', 'min', 'max'])

# Zobrazení výsledků - pro každý rok vidíme:
# - mean: průměrné srážky
# - sum: celkové srážky
# - std: směrodatnou odchylku (variabilitu srážek)
# - min: nejnižší naměřenou hodnotu
# - max: nejvyšší naměřenou hodnotu
print("Více agregačních funkcí:")
print(vysledek_vice_funkci)

# ----- 3. POJMENOVÁNÍ VÝSLEDNÝCH SLOUPCŮ -----
# Místo anglických názvů funkcí můžeme použít vlastní české názvy
# Použijeme tuple (dvojice) ve formátu: ('nový_název', 'funkce')
vysledek_pojmenovany = df_agregace.groupby('rok')['srazky'].agg([
   ('prumer', 'mean'),     # Sloupec 'prumer' bude obsahovat průměr
   ('celkem', 'sum'),      # Sloupec 'celkem' bude obsahovat součet
   ('minimum', 'min'),     # Sloupec 'minimum' bude obsahovat nejmenší hodnotu
   ('maximum', 'max')      # Sloupec 'maximum' bude obsahovat největší hodnotu
])

# Zobrazení výsledků s českými názvy sloupců
# Toto je užitečné pro vytváření reportů v češtině
print("\nPojmenované agregační funkce:")
print(vysledek_pojmenovany)

Více agregačních funkcí:
           mean   sum        std   min   max
rok                                         
2020  28.033333  84.1  15.885318  15.2  45.8
2021  33.133333  99.4  17.292869  18.7  52.3
2022  30.233333  90.7  16.086433  16.9  48.1

Pojmenované agregační funkce:
         prumer  celkem  minimum  maximum
rok                                      
2020  28.033333    84.1     15.2     45.8
2021  33.133333    99.4     18.7     52.3
2022  30.233333    90.7     16.9     48.1


### Skupinování podle více sloupců

In [28]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření slovníku s komplexnějšími daty o srážkách
# Data obsahují:
# - rok: 2021 a 2022
# - pololeti: 'h1' (první pololetí) a 'h2' (druhé pololetí)
# - mesic: konkrétní měsíc v roce (3, 6, 9, 12)
# - srazky: naměřené hodnoty srážek v mm
data_multi_groupby = {
   'rok': [2021, 2021, 2021, 2021, 2022, 2022, 2022, 2022],
   'pololeti': ['h1', 'h1', 'h2', 'h2', 'h1', 'h1', 'h2', 'h2'],
   'mesic': [3, 6, 9, 12, 3, 6, 9, 12],
   'srazky': [32.1, 45.8, 67.3, 25.7, 28.9, 42.1, 71.2, 22.4]
}

# Převod slovníku na DataFrame
df_multi_groupby = pd.DataFrame(data_multi_groupby)

# ----- 2. SKUPINOVÁNÍ PODLE VÍCE KRITÉRIÍ -----
# Pomocí seznamu ['rok', 'pololeti'] seskupíme data podle dvou sloupců současně
# To znamená, že vytvoříme skupiny pro každou kombinaci rok-pololetí:
# - 2021-h1, 2021-h2, 2022-h1, 2022-h2
multi_skupiny = df_multi_groupby.groupby(['rok', 'pololeti'])['srazky'].agg(['mean', 'sum', 'count'])

# Zobrazení výsledků
# Pro každou kombinaci rok-pololetí vidíme:
# - mean: průměrné srážky v daném pololetí
# - sum: celkové srážky v daném pololetí
# - count: počet měření v daném pololetí
print("Skupinování podle roku a pololetí:")
print(multi_skupiny)

# ----- 3. PRÁCE S MULTIINDEX VÝSLEDKY -----
# Když skupinujeme podle více sloupců, pandas vytvoří MultiIndex (víceúrovňový index)

# Zobrazení názvů jednotlivých úrovní indexu
# V našem případě: ['rok', 'pololeti']
print("\nÚrovně indexu:")
print(multi_skupiny.index.names)

# Přístup ke konkrétní kombinaci pomocí loc[]
# Používáme tuple (dvojici) pro specifikaci: (rok, pololetí)
# Například pro získání dat za rok 2021, první pololetí:
print("\nData pro rok 2021, H1:")
print(multi_skupiny.loc[(2021, 'h1')])



Skupinování podle roku a pololetí:
                mean   sum  count
rok  pololeti                    
2021 h1        38.95  77.9      2
     h2        46.50  93.0      2
2022 h1        35.50  71.0      2
     h2        46.80  93.6      2

Úrovně indexu:
['rok', 'pololeti']

Data pro rok 2021, H1:
mean     38.95
sum      77.90
count     2.00
Name: (2021, h1), dtype: float64


### Vlastní agregační funkce

In [29]:
import pandas as pd
# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření slovníku s daty o srážkách
# Data obsahují měření za dva roky (2020, 2021)
# Pro každý rok máme 3 měření (leden, červen, prosinec)
data_vlastni_funkce = {
   'rok': [2020, 2020, 2020, 2021, 2021, 2021],
   'mesic': [1, 6, 12, 1, 6, 12], 
   'srazky': [15.2, 45.8, 23.1, 18.7, 52.3, 28.4]
}

# Převod dat na DataFrame
df_vlastni_funkce = pd.DataFrame(data_vlastni_funkce)

# ----- 2. DEFINICE VLASTNÍCH FUNKCÍ -----
# Kromě vestavěných funkcí (mean, sum, atd.) můžeme vytvořit vlastní

def vypocti_rozpeti(serie):
   """Výpočet rozpětí hodnot (max - min)"""
   # Funkce přijímá pandas Series a vrací rozdíl mezi nejvyšší a nejnižší hodnotou
   return serie.max() - serie.min()

def vypocti_koeficient_variace(serie):
   """Výpočet koeficientu variace v procentech"""
   # Koeficient variace = (směrodatná odchylka / průměr) * 100
   # Udává relativní variabilitu dat v procentech
   return (serie.std() / serie.mean()) * 100

# ----- 3. APLIKACE VLASTNÍCH FUNKCÍ -----
# V metodě agg() můžeme použít naše vlastní funkce
# Stačí předat název funkce (bez závorek)
vlastni_statistiky = df_vlastni_funkce.groupby('rok')['srazky'].agg([
   ('prumer', 'mean'),                          # Vestavěná funkce pro průměr
   ('rozpeti', vypocti_rozpeti),               # Naše funkce pro rozpětí
   ('koef_variace', vypocti_koeficient_variace) # Naše funkce pro koef. variace
])

# Zobrazení výsledků s vlastními výpočty
print("Vlastní agregační funkce:")
print(vlastni_statistiky)

# ----- 4. POUŽITÍ LAMBDA FUNKCÍ -----
# Lambda funkce jsou anonymní funkce definované přímo v kódu
# Hodí se pro jednoduché operace, které nepotřebujeme znovu použít

lambda_vysledek = df_vlastni_funkce.groupby('rok')['srazky'].agg([
   ('prumer', 'mean'),                                    # Standardní průměr
   ('pocet_nad_prumer', lambda x: (x > x.mean()).sum())  # Lambda funkce
   # Lambda funkce vysvětlena:
   # - x: pandas Series se srážkami pro daný rok
   # - x > x.mean(): vytvoří boolean Series (True/False)
   # - .sum(): sečte True hodnoty (True = 1, False = 0)
   # Výsledek: počet měsíců s nadprůměrnými srážkami
])

# Zobrazení výsledků s lambda funkcí
print("\nLambda funkce:")
print(lambda_vysledek)

Vlastní agregační funkce:
         prumer  rozpeti  koef_variace
rok                                   
2020  28.033333     30.6     56.665820
2021  33.133333     33.6     52.191759

Lambda funkce:
         prumer  pocet_nad_prumer
rok                              
2020  28.033333                 1
2021  33.133333                 1


### Transform operace

Transform zachovává původní velikost DataFrame na rozdíl od agregace.

In [30]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření dat o srážkách ze dvou meteorologických stanic
# Každá stanice má měření za 3 měsíce (duben, květen, červen)
data_transform = {
   'stanice': ['praha', 'praha', 'praha', 'brno', 'brno', 'brno'],
   'mesic': [4, 5, 6, 4, 5, 6],
   'srazky': [28.5, 42.7, 35.1, 31.2, 38.9, 33.6]
}

# Převod na DataFrame
df_transform = pd.DataFrame(data_transform)

# ----- 2. TRANSFORM VS AGG - DŮLEŽITÝ ROZDÍL -----
# agg(): vrací agregovaný výsledek (zmenší počet řádků)
# transform(): vrací výsledek stejné velikosti jako původní data

# ----- 3. PŘIDÁNÍ PRŮMĚRU PODLE STANICE -----
# Pro každý řádek přidáme průměr srážek dané stanice
# Výsledek má stejný počet řádků jako původní data
df_transform['prumer_stanice'] = df_transform.groupby('stanice')['srazky'].transform('mean')
# Praha: všechny 3 řádky budou mít hodnotu průměru Prahy
# Brno: všechny 3 řádky budou mít hodnotu průměru Brna

# ----- 4. ROZDÍL OD PRŮMĚRU -----
# Pro každý měsíc vypočítáme, o kolik se liší od průměru své stanice
# Kladná hodnota = nadprůměrné srážky, záporná = podprůměrné
df_transform['rozdil_od_prumeru'] = df_transform.groupby('stanice')['srazky'].transform(
   lambda x: x - x.mean()
)
# Například: pokud Praha má průměr 35.4 mm a v dubnu bylo 28.5 mm,
# pak rozdíl bude -6.9 mm (bylo méně srážek než obvykle)

# ----- 5. POŘADÍ V RÁMCI STANICE -----
# Pro každou stanici seřadíme měsíce podle srážek
# 1 = nejméně srážek, 3 = nejvíce srážek
df_transform['poradi_ve_stanici'] = df_transform.groupby('stanice')['srazky'].transform(
   lambda x: x.rank()
)
# Každý měsíc dostane číslo 1, 2 nebo 3 podle toho,
# kolik měl srážek ve srovnání s ostatními měsíci na stejné stanici

# ----- 6. ZOBRAZENÍ VÝSLEDKŮ -----
# Všimněte si, že všechny nové sloupce mají stejný počet řádků jako původní data
print("Transform operace:")
print(df_transform)

# Poznámka: Transform je velmi užitečný pro:
# - Přidání skupinových statistik ke každému řádku
# - Výpočet rozdílů od skupinového průměru
# - Určení pořadí v rámci skupin

Transform operace:
  stanice  mesic  srazky  prumer_stanice  rozdil_od_prumeru  poradi_ve_stanici
0   praha      4    28.5       35.433333          -6.933333                1.0
1   praha      5    42.7       35.433333           7.266667                3.0
2   praha      6    35.1       35.433333          -0.333333                2.0
3    brno      4    31.2       34.566667          -3.366667                1.0
4    brno      5    38.9       34.566667           4.333333                3.0
5    brno      6    33.6       34.566667          -0.966667                2.0


## Pivot transformace

### Základní `df.pivot()`

Pivot přeformátuje data z dlouhého formátu na široký. Vhodné pro data bez duplicitních kombinací.

In [31]:
import pandas as pd
# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření dat o srážkách za 3 roky
# Pro každý rok máme měření ze dvou měsíců (červen a prosinec)
data_pivot = {
   'rok': [2021, 2021, 2022, 2022, 2023, 2023],
   'mesic': [6, 12, 6, 12, 6, 12],
   'srazky': [45.2, 28.7, 42.1, 31.4, 48.9, 25.6]
}

# Převod na DataFrame
df_pivot = pd.DataFrame(data_pivot)

# Zobrazení původních dat v "dlouhém" formátu
# Každý řádek = jedno měření
print("Původní data:")
print(df_pivot)

# ----- 2. PIVOT TRANSFORMACE -----
# Pivot převádí data z "dlouhého" formátu na "široký" formát
# Je to jako vytvoření kontingenční tabulky
pivot_vysledek = df_pivot.pivot(
   index='rok',        # Co bude v řádcích (roky: 2021, 2022, 2023)
   columns='mesic',    # Co bude ve sloupcích (měsíce: 6, 12)
   values='srazky'     # Jaké hodnoty budou v buňkách tabulky
)

# Výsledkem je tabulka, kde:
# - každý řádek = jeden rok
# - každý sloupec = jeden měsíc
# - v buňkách jsou hodnoty srážek
print("\nPo pivot transformaci:")
print(pivot_vysledek)

# ----- 3. ÚPRAVA FORMÁTOVÁNÍ -----
# Po pivotu má osa sloupců název "mesic", což může být rušivé
# Odstraníme tento název pro čistší vzhled
pivot_vysledek.columns.name = None  # Odstranění názvu osy sloupců

# Zobrazení finální tabulky
print("\nPo úpravě názvů:")
print(pivot_vysledek)

# Poznámka: Pivot je užitečný pro:
# - Převod dat do formátu vhodného pro analýzu časových řad
# - Vytváření přehledných tabulek pro reporty
# - Přípravu dat pro vizualizace (heatmapy, grafy)

Původní data:
    rok  mesic  srazky
0  2021      6    45.2
1  2021     12    28.7
2  2022      6    42.1
3  2022     12    31.4
4  2023      6    48.9
5  2023     12    25.6

Po pivot transformaci:
mesic    6     12
rok              
2021   45.2  28.7
2022   42.1  31.4
2023   48.9  25.6

Po úpravě názvů:
        6     12
rok             
2021  45.2  28.7
2022  42.1  31.4
2023  48.9  25.6


### Pokročilé pivot operace

In [32]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření dat se dvěma metrikami: srážky a teplota
# Data obsahují měření pro léto a zimu v letech 2021 a 2022
data_multi_pivot = {
   'rok': [2021, 2021, 2022, 2022],
   'sezona': ['leto', 'zima', 'leto', 'zima'],
   'srazky': [145.8, 87.2, 152.3, 92.7],
   'teplota': [20.5, 2.8, 21.2, 1.9]
}

# Převod na DataFrame
df_multi_pivot = pd.DataFrame(data_multi_pivot)

# ----- 2. PIVOT S VÍCE HODNOTAMI -----
# Když zadáme seznam hodnot ['srazky', 'teplota'], 
# pivot vytvoří hierarchické (víceúrovňové) sloupce
multi_pivot_vysledek = df_multi_pivot.pivot(
   index='rok',                        # Řádky: roky 2021, 2022
   columns='sezona',                   # Sloupce: leto, zima  
   values=['srazky', 'teplota']      # Seznam hodnot pro pivot
)

# Výsledná tabulka má dvouúrovňové sloupce:
# 1. úroveň: metrika (srazky, teplota)
# 2. úroveň: sezona (leto, zima)
print("Pivot s více hodnotami:")
print(multi_pivot_vysledek)

# ----- 3. PRÁCE S HIERARCHICKÝMI SLOUPCI -----
# Pro přístup k hodnotám v hierarchických sloupcích
# používáme tuple (dvojici) ve formátu: (metrika, sezona)
print("\nPřístup k srážkám v létě:")
print(multi_pivot_vysledek[('srazky', 'leto')])

# Tímto způsobem získáme Series se srážkami v létě
# pro všechny roky (2021: 145.8, 2022: 152.3)

Pivot s více hodnotami:
       srazky       teplota     
sezona   leto  zima    leto zima
rok                             
2021    145.8  87.2    20.5  2.8
2022    152.3  92.7    21.2  1.9

Přístup k srážkám v létě:
rok
2021    145.8
2022    152.3
Name: (srazky, leto), dtype: float64


In [33]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT S NEÚPLNÝMI ZÁZNAMY -----
# Vytvoření dat, kde rok 2022 má měření pouze pro březen
# Roky 2020 a 2021 mají měření pro březen i září
data_chybejici = {
   'rok': [2020, 2020, 2021, 2021, 2022],  # 2022 má jen jedno měření
   'mesic': [3, 9, 3, 9, 3],
   'srazky': [28.4, 65.7, 31.2, 68.9, 29.8]
}

# Převod na DataFrame
df_chybejici = pd.DataFrame(data_chybejici)

# ----- 2. PIVOT S CHYBĚJÍCÍMI HODNOTAMI -----
# Když některé kombinace rok-měsíc neexistují v datech,
# pivot automaticky vloží NaN (Not a Number) na místa chybějících hodnot
pivot_s_nan = df_chybejici.pivot(
   index='rok',        # Řádky: 2020, 2021, 2022
   columns='mesic',    # Sloupce: 3, 9
   values='srazky'     # Hodnoty srážek
)

# Všimněte si, že rok 2022 má NaN v sloupci 9 (září),
# protože pro tuto kombinaci nemáme data
print("Pivot s chybějícími hodnotami:")
print(pivot_s_nan)

# ----- 3. VYPLNĚNÍ CHYBĚJÍCÍCH HODNOT -----
# fillna() nahradí všechny NaN hodnoty zadanou hodnotou
# Zde nahrazujeme chybějící měření nulou
pivot_vyplneny = pivot_s_nan.fillna(0)

# Výsledek: místo NaN je nyní 0 pro září 2022
print("\nPo vyplnění nulami:")
print(pivot_vyplneny)

# Alternativní možnosti vyplnění:
# fillna(pivot_s_nan.mean()) - vyplní průměrem sloupce
# fillna(method='ffill') - vyplní předchozí hodnotou
# fillna(method='bfill') - vyplní následující hodnotou

Pivot s chybějícími hodnotami:
mesic     3     9
rok              
2020   28.4  65.7
2021   31.2  68.9
2022   29.8   NaN

Po vyplnění nulami:
mesic     3     9
rok              
2020   28.4  65.7
2021   31.2  68.9
2022   29.8   0.0


## Pivot Table

### Základní `df.pivot_table()`

Pivot table automaticky agreguje duplicitní kombinace klíčů.

In [34]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT S VÍCE HODNOTAMI PRO STEJNOU KOMBINACI -----
# Data obsahují měření ze dvou stanic (Praha, Brno) pro každé pololetí
# Pro každou kombinaci rok-pololetí máme tedy 2 hodnoty
data_pivot_table = {
   'rok': [2020, 2020, 2020, 2020, 2021, 2021, 2021, 2021],
   'pololeti': ['h1', 'h1', 'h2', 'h2', 'h1', 'h1', 'h2', 'h2'],
   'stanice': ['praha', 'brno', 'praha', 'brno', 'praha', 'brno', 'praha', 'brno'],
   'srazky': [285.4, 267.8, 298.7, 312.1, 301.2, 289.6, 276.8, 295.4]
}

# Převod na DataFrame
df_pivot_table = pd.DataFrame(data_pivot_table)

# Zobrazení všech dat - vidíme, že pro každou kombinaci rok-pololetí
# máme dvě měření (Praha a Brno)
print("Data s více stanicemi:")
print(df_pivot_table)

# ----- 2. PIVOT_TABLE MÍSTO PIVOT -----
# Když máme duplicitní kombinace (rok-pololetí), 
# nemůžeme použít pivot(), protože by nevěděl, kterou hodnotu vybrat
# pivot_table() automaticky agreguje (seskupí) duplicitní hodnoty

# ----- 3. VYTVOŘENÍ PIVOT TABLE S PRŮMĚREM -----
tabulka_prumer = df_pivot_table.pivot_table(
   index='rok',         # Řádky: 2020, 2021
   columns='pololeti',  # Sloupce: h1, h2
   values='srazky'      # Hodnoty k agregaci
   # aggfunc='mean' je výchozí, nemusíme zadávat
)

# Výsledná tabulka obsahuje průměr srážek z Prahy a Brna
# pro každou kombinaci rok-pololetí
print("\nPivot table s průměrem:")
print(tabulka_prumer)

# Například hodnota pro rok 2020, h1:
# (285.4 + 267.8) / 2 = 276.6

Data s více stanicemi:
    rok pololeti stanice  srazky
0  2020       h1   praha   285.4
1  2020       h1    brno   267.8
2  2020       h2   praha   298.7
3  2020       h2    brno   312.1
4  2021       h1   praha   301.2
5  2021       h1    brno   289.6
6  2021       h2   praha   276.8
7  2021       h2    brno   295.4

Pivot table s průměrem:
pololeti     h1     h2
rok                   
2020      276.6  305.4
2021      295.4  286.1


### Pokročilé pivot table

In [35]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Data obsahují srážky pro dva regiony (Morava, Čechy) 
# ve dvou měsících (březen, září) za dva roky
data_komplexni_pivot = {
   'rok': [2021, 2021, 2021, 2021, 2022, 2022, 2022, 2022],
   'region': ['morava', 'morava', 'cechy', 'cechy', 'morava', 'morava', 'cechy', 'cechy'],
   'mesic': [3, 9, 3, 9, 3, 9, 3, 9],
   'srazky': [32.4, 68.7, 29.8, 71.2, 35.1, 65.9, 31.7, 73.5]
}

# Převod na DataFrame
df_komplexni_pivot = pd.DataFrame(data_komplexni_pivot)

# ----- 2. KOMPLEXNÍ PIVOT TABLE S VÍCE FUNKCEMI -----
# Vytvoříme tabulku s více agregačními funkcemi najednou
komplexni_tabulka = df_komplexni_pivot.pivot_table(
   index='rok',                           # Řádky: 2021, 2022
   columns='region',                      # Sloupce: cechy, morava
   values='srazky',                       # Hodnoty k agregaci
   aggfunc=['mean', 'sum', 'count'],     # Seznam funkcí k použití
   margins=True,                          # Přidá řádek a sloupec se souhrny
   margins_name='celkem'                  # Název pro souhrnný řádek/sloupec
)

# Výsledná tabulka má hierarchické sloupce:
# - 1. úroveň: agregační funkce (mean, sum, count)
# - 2. úroveň: regiony (cechy, morava, celkem)
# Řádek "celkem" obsahuje souhrny napříč všemi roky
print("Komplexní pivot table:")
print(komplexni_tabulka)

# ----- 3. JEDNODUŠŠÍ VERZE BEZ SOUHRNŮ -----
# Alternativní způsob zadání funkcí pomocí slovníku
jednoducha_tabulka = df_komplexni_pivot.pivot_table(
   index='rok',                           # Řádky: 2021, 2022
   columns='region',                      # Sloupce: cechy, morava
   values='srazky',                       # Hodnoty k agregaci
   aggfunc={'srazky': ['mean', 'sum']}   # Slovník: {sloupec: [funkce]}
)

# Tato verze neobsahuje řádek "celkem" a má pouze dvě funkce
print("\nJednodušší pivot table:")
print(jednoducha_tabulka)

Komplexní pivot table:
         mean                     sum               count              
region  cechy  morava   celkem  cechy morava celkem cechy morava celkem
rok                                                                    
2021    50.50  50.550  50.5250  101.0  101.1  202.1     2      2      4
2022    52.60  50.500  51.5500  105.2  101.0  206.2     2      2      4
celkem  51.55  50.525  51.0375  206.2  202.1  408.3     4      4      8

Jednodušší pivot table:
        mean           sum       
region cechy morava  cechy morava
rok                              
2021    50.5  50.55  101.0  101.1
2022    52.6  50.50  105.2  101.0


### Kompletní přehled parametrů

| Parametr | Popis | Výchozí hodnota |
|----------|-------|-----------------|
| `index` | Sloupce pro řádky | None |
| `columns` | Sloupce pro sloupce | None |
| `values` | Sloupce pro agregaci | None |
| `aggfunc` | Agregační funkce | 'mean' |
| `margins` | Přidání součtových řádků/sloupců | False |
| `margins_name` | Název pro margins | 'All' |
| `fill_value` | Hodnota pro chybějící data | None |

In [36]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Data obsahují srážky pro první a třetí čtvrtletí za tři roky
# Všimněte si, že nemáme data pro Q2 a Q4
data_parametry = {
   'rok': [2021, 2021, 2022, 2022, 2023, 2023],
   'ctvrtleti': ['q1', 'q3', 'q1', 'q3', 'q1', 'q3'],
   'srazky': [78.5, 145.2, 82.1, 138.7, 75.9, 149.3]
}

# Převod na DataFrame
df_parametry = pd.DataFrame(data_parametry)

# ----- 2. VYTVOŘENÍ PIVOT TABLE S RŮZNÝMI PARAMETRY -----
tabulka_parametry = df_parametry.pivot_table(
   index='rok',                    # Řádky: 2021, 2022, 2023
   columns='ctvrtleti',           # Sloupce: q1, q3
   values='srazky',               # Hodnoty srážek
   aggfunc='sum',                 # Sečteme hodnoty (zde vlastně není co sčítat)
   fill_value=0,                  # Místo NaN doplníme 0
   margins=True,                  # Přidáme souhrnný řádek a sloupec
   margins_name='rocni_soucet'    # Pojmenujeme souhrnný řádek/sloupec
)

# ----- 3. VYSVĚTLENÍ VÝSLEDKU -----
# Tabulka obsahuje:
# - Sloupce: q1, q3, rocni_soucet
# - Řádky: 2021, 2022, 2023, rocni_soucet
# - Sloupec "rocni_soucet" = součet Q1 + Q3 pro každý rok
# - Řádek "rocni_soucet" = součet všech let pro každé čtvrtletí
# - Hodnota vpravo dole = celkový součet všech srážek

print("Pivot table s parametry:")
print(tabulka_parametry)

Pivot table s parametry:
ctvrtleti        q1     q3  rocni_soucet
rok                                     
2021           78.5  145.2         223.7
2022           82.1  138.7         220.8
2023           75.9  149.3         225.2
rocni_soucet  236.5  433.2         669.7


## Transpose operace

### Základní `df.transpose()`

Transpose vymění řádky a sloupce DataFrame.

In [37]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Vytvoření DataFrame, kde:
# - sloupce = měsíce (leden, únor, březen)
# - řádky = roky (2021, 2022, 2023)
# - hodnoty = srážky v mm
data_transpose = {
   'leden': [15.2, 18.7, 22.1],
   'unor': [12.8, 16.4, 19.9], 
   'brezen': [28.5, 31.2, 35.7]
}

# Vytvoření DataFrame s pojmenovaným indexem (roky)
df_transpose = pd.DataFrame(data_transpose, index=['2021', '2022', '2023'])

# Zobrazení původní struktury
print("Původní DataFrame:")
print(df_transpose)

# ----- 2. TRANSPOSE POMOCÍ .T -----
# .T je zkrácený zápis pro transpozici
# Prohodí řádky a sloupce
df_transponovany_t = df_transpose.T

# Po transpozici:
# - sloupce = roky (2021, 2022, 2023)
# - řádky = měsíce (leden, únor, březen)
print("\nTranspose (.T):")
print(df_transponovany_t)

# ----- 3. TRANSPOSE POMOCÍ .transpose() -----
# .transpose() je plný název metody, dělá totéž co .T
df_transponovany_metoda = df_transpose.transpose()

print("\nTranspose (.transpose()):")
print(df_transponovany_metoda)

# ----- 4. OVĚŘENÍ, ŽE OBA ZPŮSOBY JSOU IDENTICKÉ -----
# .equals() porovná dva DataFramy a vrátí True, pokud jsou stejné
print(f"\nShodné výsledky: {df_transponovany_t.equals(df_transponovany_metoda)}")

Původní DataFrame:
      leden  unor  brezen
2021   15.2  12.8    28.5
2022   18.7  16.4    31.2
2023   22.1  19.9    35.7

Transpose (.T):
        2021  2022  2023
leden   15.2  18.7  22.1
unor    12.8  16.4  19.9
brezen  28.5  31.2  35.7

Transpose (.transpose()):
        2021  2022  2023
leden   15.2  18.7  22.1
unor    12.8  16.4  19.9
brezen  28.5  31.2  35.7

Shodné výsledky: True


### Praktické aplikace transpose

In [38]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Data obsahují srážky po čtvrtletích pro tři roky
# Struktura: jeden řádek = jeden rok, sloupce = čtvrtletí
data_ctvrtleti = {
   'rok': [2021, 2022, 2023],
   'q1_srazky': [78.4, 82.1, 75.8],
   'q2_srazky': [145.7, 138.9, 152.3],
   'q3_srazky': [167.2, 171.6, 163.4],
   'q4_srazky': [89.7, 93.2, 87.1]
}

# Převod na DataFrame
df_ctvrtleti = pd.DataFrame(data_ctvrtleti)

# Zobrazení původní struktury
print("Původní čtvrtletní data:")
print(df_ctvrtleti)

# ----- 2. PŘÍPRAVA NA TRANSPOSE -----
# Před transpozicí nastavíme sloupec 'rok' jako index
# Jinak by se 'rok' stal běžným řádkem po transpozici
df_s_indexem = df_ctvrtleti.set_index('rok')

# Provedení transpozice
df_transponovane_ctvrtleti = df_s_indexem.T

# Po transpozici:
# - řádky = čtvrtletí (q1_srazky, q2_srazky, atd.)
# - sloupce = roky (2021, 2022, 2023)
print("\nData po transpose:")
print(df_transponovane_ctvrtleti)

# ----- 3. ÚPRAVA NÁZVŮ PRO LEPŠÍ ČITELNOST -----
# Původní názvy řádků (q1_srazky, q2_srazky) nejsou ideální
# Změníme je na jednodušší Q1, Q2, Q3, Q4
df_transponovane_ctvrtleti.index = ['Q1', 'Q2', 'Q3', 'Q4']

# Finální podoba - přehledná tabulka čtvrtletí × roky
print("\nData s upravenými názvy čtvrtletí:")
print(df_transponovane_ctvrtleti)

Původní čtvrtletní data:
    rok  q1_srazky  q2_srazky  q3_srazky  q4_srazky
0  2021       78.4      145.7      167.2       89.7
1  2022       82.1      138.9      171.6       93.2
2  2023       75.8      152.3      163.4       87.1

Data po transpose:
rok         2021   2022   2023
q1_srazky   78.4   82.1   75.8
q2_srazky  145.7  138.9  152.3
q3_srazky  167.2  171.6  163.4
q4_srazky   89.7   93.2   87.1

Data s upravenými názvy čtvrtletí:
rok   2021   2022   2023
Q1    78.4   82.1   75.8
Q2   145.7  138.9  152.3
Q3   167.2  171.6  163.4
Q4    89.7   93.2   87.1


## Kombinování transformací

In [39]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Data obsahují srážky za 3 roky, vždy pro leden, červen a prosinec
data_kombinace = {
   'rok': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022],
   'mesic': [1, 6, 12, 1, 6, 12, 1, 6, 12],
   'srazky': [18.5, 45.2, 22.7, 21.3, 42.8, 25.1, 19.7, 48.6, 23.9]
}

# Převod na DataFrame
df_kombinace = pd.DataFrame(data_kombinace)

# Zobrazení původních dat v "dlouhém" formátu
print("Původní data pro kombinované transformace:")
print(df_kombinace)

# ----- 2. KROK 1: PIVOT PRO PŘEFORMÁTOVÁNÍ -----
# Změníme strukturu dat tak, aby:
# - řádky = měsíce (1, 6, 12)
# - sloupce = roky (2020, 2021, 2022)
pivot_kombinace = df_kombinace.pivot(
   index='mesic',      # Měsíce budou v řádcích
   columns='rok',      # Roky budou ve sloupcích
   values='srazky'     # Hodnoty srážek v buňkách
)

print("\n1. Po pivot transformaci:")
print(pivot_kombinace)

# ----- 3. KROK 2: PŘIDÁNÍ STATISTIK -----
# K tabulce přidáme nové sloupce s výpočty

# Průměr srážek pro každý měsíc napříč všemi roky
# axis=1 znamená počítat po řádcích (horizontálně)
pivot_kombinace['prumer_mesice'] = pivot_kombinace.mean(axis=1)

# Maximum srážek pro každý měsíc napříč všemi roky
pivot_kombinace['max_mesice'] = pivot_kombinace.max(axis=1)

print("\n2. Po přidání statistik:")
print(pivot_kombinace)

# ----- 4. KROK 3: TRANSPOSE PRO JINÝ POHLED -----
# Prohodíme řádky a sloupce pro jiný pohled na data
# Teď budou roky a statistiky v řádcích, měsíce ve sloupcích
finalni_pohled = pivot_kombinace.T

print("\n3. Finální pohled po transpose:")
print(finalni_pohled)

# Výsledná tabulka ukazuje:
# - v řádcích: roky (2020-2022) + vypočtené statistiky
# - ve sloupcích: měsíce (1, 6, 12)
# Tento pohled je užitečný pro porovnání trendů napříč měsíci

Původní data pro kombinované transformace:
    rok  mesic  srazky
0  2020      1    18.5
1  2020      6    45.2
2  2020     12    22.7
3  2021      1    21.3
4  2021      6    42.8
5  2021     12    25.1
6  2022      1    19.7
7  2022      6    48.6
8  2022     12    23.9

1. Po pivot transformaci:
rok    2020  2021  2022
mesic                  
1      18.5  21.3  19.7
6      45.2  42.8  48.6
12     22.7  25.1  23.9

2. Po přidání statistik:
rok    2020  2021  2022  prumer_mesice  max_mesice
mesic                                             
1      18.5  21.3  19.7      19.833333        21.3
6      45.2  42.8  48.6      45.533333        48.6
12     22.7  25.1  23.9      23.900000        25.1

3. Finální pohled po transpose:
mesic                 1          6     12
rok                                      
2020           18.500000  45.200000  22.7
2021           21.300000  42.800000  25.1
2022           19.700000  48.600000  23.9
prumer_mesice  19.833333  45.533333  23.9
max_mesice    

In [40]:
import pandas as pd

# ----- 1. PŘÍPRAVA DAT -----
# Data obsahují srážky ze dvou stanic (Praha, Brno) 
# za dvě pololetí (h1, h2) ve dvou letech (2021, 2022)
data_komplexni_priklad = {
   'rok': [2021, 2021, 2021, 2021, 2022, 2022, 2022, 2022],
   'stanice': ['praha', 'brno', 'praha', 'brno', 'praha', 'brno', 'praha', 'brno'],
   'pololeti': ['h1', 'h1', 'h2', 'h2', 'h1', 'h1', 'h2', 'h2'],
   'srazky': [234.5, 221.8, 267.3, 289.6, 241.2, 228.7, 275.1, 295.4]
}

# Převod na DataFrame
df_komplexni_priklad = pd.DataFrame(data_komplexni_priklad)

# Zobrazení původních dat
print("Komplexní příklad - původní data:")
print(df_komplexni_priklad)

# ----- 2. KROK 1: GROUPBY PRO AGREGACI -----
# Seskupíme data podle stanice a pololetí
# Vypočítáme průměr srážek pro každou kombinaci
skupiny_stanice_pololeti = (
   df_komplexni_priklad
   .groupby(['stanice', 'pololeti'])['srazky']  # Seskupení podle 2 kritérií
   .mean()                                       # Výpočet průměru
   .reset_index()                                # Převod indexu zpět na sloupce
)

# Výsledek: průměr srážek pro každou kombinaci stanice-pololetí
# (průměr ze dvou let 2021 a 2022)
print("\n1. Po GroupBy agregaci:")
print(skupiny_stanice_pololeti)

# ----- 3. KROK 2: PIVOT PRO PŘEHLEDNÉ ZOBRAZENÍ -----
# Převedeme agregovaná data do tabulkové formy
finalni_tabulka = skupiny_stanice_pololeti.pivot(
   index='stanice',     # Stanice budou v řádcích
   columns='pololeti',  # Pololetí budou ve sloupcích
   values='srazky'      # Průměrné srážky v buňkách
)

print("\n2. Finální tabulka po pivot:")
print(finalni_tabulka)

# ----- 4. KROK 3: PŘIDÁNÍ CELKOVÝCH STATISTIK -----
# Přidáme sloupec s průměrem pro každou stanici
# axis=1 znamená počítat průměr po řádku (z h1 a h2)
finalni_tabulka['prumer_stanice'] = finalni_tabulka.mean(axis=1)

# Přidáme řádek s průměrem pro každé pololetí
# .loc['prumer_pololeti'] vytvoří nový řádek s tímto názvem
# Bez axis parametru se počítá průměr po sloupcích (vertikálně)
finalni_tabulka.loc['prumer_pololeti'] = finalni_tabulka.mean()

# Výsledná tabulka obsahuje:
# - průměrné srážky pro každou kombinaci stanice-pololetí
# - průměr za celou stanici (sloupec vpravo)
# - průměr za celé pololetí (řádek dole)
print("\n3. S celkovými statistikami:")
print(finalni_tabulka)

Komplexní příklad - původní data:
    rok stanice pololeti  srazky
0  2021   praha       h1   234.5
1  2021    brno       h1   221.8
2  2021   praha       h2   267.3
3  2021    brno       h2   289.6
4  2022   praha       h1   241.2
5  2022    brno       h1   228.7
6  2022   praha       h2   275.1
7  2022    brno       h2   295.4

1. Po GroupBy agregaci:
  stanice pololeti  srazky
0    brno       h1  225.25
1    brno       h2  292.50
2   praha       h1  237.85
3   praha       h2  271.20

2. Finální tabulka po pivot:
pololeti      h1     h2
stanice                
brno      225.25  292.5
praha     237.85  271.2

3. S celkovými statistikami:
pololeti             h1      h2  prumer_stanice
stanice                                        
brno             225.25  292.50         258.875
praha            237.85  271.20         254.525
prumer_pololeti  231.55  281.85         256.700


## Shrnutí

- **GroupBy** je základní nástroj pro agregace podle skupin.
- **Pivot** transformuje data z dlouhého na široký formát.
- **Pivot table** přidává agregační funkce při duplicitních hodnotách.
- **Transpose** jednoduše otáčí řádky a sloupce.
- Tyto metody lze kombinovat pro komplexní transformace dat.